In [1]:
import numpy as np
import pandas as pd
import itertools


from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

### 전처리가 이미 완료된 후의 데이터를 사용
보안상 데이터를 전체 공개 불가

In [2]:
df = pd.read_csv('../data/dataset/CAI_data_003.csv')

### 불용어 처리 

In [ ]:
stop_words = "고객 요청사항 요청사 항 본인 긴급 독촉심함 독촉 봇 빠른 방문 요청 요청함 중 회선단말특이사항 단말특이사항 불 후 정상 측 연락처 안 됨기사 안됨 추가 특이사항 서비스 회선 진단 결과 코로나 기타"

In [ ]:
preprocess_data = preprocess_data.reset_index(drop= True)
aa = []
for i in range(len(preprocess_data)) :
    aa.append(preprocess_data['sentence'][i].split(' '))
aa

In [3]:
df['sentence'][1]

'고객 요청사항 수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일중 꼭 점검원하여 지사 이관함회선 단말특이사항 진단 결과 기타 추가 연락처'

In [4]:
okt = Okt()

tokenized_doc = okt.pos(df['sentence'][1])
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] != 0])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('고객', 'Noun'), ('요청', 'Noun'), ('사항', 'Noun'), ('수', 'Modifier'), ('신', 'Modifier'), ('불', 'Noun'), ('인터넷', 'Noun'), ('연결', 'Noun'), ('오류', 'Noun'), ('뜸', 'Noun')]
추출 : 고객 요청 사항 수 신 불 인터넷 연결 오류 뜸 재부팅 해 봄 모텔 영업 용 일 중 꼭 점검 원하여 지사 이 관함 회선 단말 특이 사항 진단 결과 기타 추가 연락처


In [5]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 41
trigram 다섯개만 출력 : ['결과 기타' '결과 기타 추가' '고객 요청' '고객 요청 사항' '관함 회선']


In [6]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([df['sentence'][1]])
candidate_embeddings = model.encode(candidates)

In [7]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['점검 원하여', '연결 오류 재부팅', '원하여 지사 관함', '점검 원하여 지사', '영업 점검 원하여']


In [8]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings,
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None

    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
abstract_sen = []
for i in range(len(df)) :
    try :
        okt = Okt()

        tokenized_doc = okt.pos(df['sentence'][i])
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] != 0])

        n_gram_range = (2, 3)

        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
        candidates = count.get_feature_names_out()

        doc_embedding = model.encode([df['sentence'][i]])
        candidate_embeddings = model.encode(candidates)

        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
        
        
        keywords = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=3, nr_candidates=10)
        
        
        abstract_sen.append(keywords)
        print('----------------------------------------------------------')
        print('원본 : ', df['sentence'][i])
        print('요약 : ', abstract_sen[i])
        print('----------------------------------------------------------')

    except :
        abstract_sen.append(df['sentence'][i])



In [ ]:
abstract_sen

## Max Sum Similarity
데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. 여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.